This is attempt to fix lstm.ipynb notebook with multilabel classification + Poisson regression at each timestep. I was hoping that repetitive character of the text is making problems. But it turned out that it was somethign different. The problem was again the same. Very fast convergence to returning the same and the same values again. I had also some problem with exploding values for the Poisson Loss, which was probably also due to processing sequences one by 1.
Warning, this notebook is quite messy.

In [428]:
import sys
sys.path.append("D:\\GhostPresentation")
%load_ext autoreload
%autoreload 2

from midiToTxt import converter
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from tqdm import tqdm
from torch import optim
import pypianoroll
import random
import numpy as np



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [429]:
START_TOKEN = "<START>" 
END_TOKEN = "<END>"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
TEACHER_FORCING_RATIO = 0.5 #Whether or not to use teaacher forcing method 
POSITIVE_WEIGHT = 50

In [430]:
class midiTextGeneratorDataset(Dataset):
    def __init__(self, path:str):
        with open(path, 'r') as f:
            corpus = f.read().strip()
        
        #In that case we must filter everything at first and divide tokens and counts
        self.data = corpus.split("\n")
        self.tokens = []
        self.counts = []
        
        for i, song in enumerate(self.data):
            self.tokens.append([])
            self.counts.append([])
            
            tokenized = song.split(" ")
            
            for j in range(0,len(tokenized)):
                if j % 2 == 0:
                    self.tokens[i].append(tokenized[j])
                else:
                    self.counts[i].append(int(tokenized[j]))
        #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        self.unique_letters = set("".join(["".join(song) for song in self.tokens]))
       
        #self.unique_letters.add(END_TOKEN)
        #self.unique_letters.add(START_TOKEN)
        
        self.num2word = self.get_num2word_mapping(self.unique_letters)
        self.word2num = {v:k for k,v in self.num2word.items()}
        
        self.tensor_size = len(self.word2num)
        
    
    def get_num2word_mapping(self, letters: set) -> dict:
        mapping = {}
        for index, l in enumerate(letters):
            mapping[index] = l
            
        return mapping
    
    def token_to_tensor(self, token: str) -> torch.tensor:
        tensor = torch.zeros((self.tensor_size, 1))
        # if token == "":
        #     return tensor
        
        if token == START_TOKEN:
            tensor[self.word2num[START_TOKEN]] = 1
        
        elif token == END_TOKEN:
            tensor[self.word2num[END_TOKEN]] = 1
        
        elif token != "":  
            for letter in token:
                tensor[self.word2num[letter]] = 1
        
        return torch.unsqueeze(tensor.view(1, -1),0) # we want [1, 1, num_of_characters]
    
    def tensor_to_token(self, tensor: torch.tensor) -> str:
        notes = tensor.nonzero()
        token = ""
        for _, note in notes:
            token += self.num2word[note.item()]
            if len(token) > 10:
                break
            
        return token
    
    def __len__(self) -> int:
        return len(self.data)
    
    def __getitem__(self, idx):
        song = list(self.tokens[idx]) #MUST DO A COPY HERE SINCE WE THEN MODIFY THAT ARRAY!!!
        
        #song.insert(0, START_TOKEN)
        #song.append(END_TOKEN)
        
        inp = self.token_to_tensor(song[0])
        #print(inp.shape)
        for token in song[1:]:
            t = self.token_to_tensor(token)
            #print(t.shape)
            inp = torch.cat((inp, t), dim=0)
        
        counts = list(self.counts[idx])
        #counts.insert(0, 0)
        #counts.append(0)
        
        #print(song)
        #print(counts)
        #self.print_given_song(idx)
        
        return inp, torch.tensor(counts)
    
    def print_given_song(self, idx):
        print(len(self.data[idx].split(" ")))
        print(self.data[idx])

    
        

In [423]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.gru = nn.GRU(output_size + 1, hidden_size) # since we have notes played + number of times it happened
        self.out_classification = nn.Linear(hidden_size, output_size)
        self.out_counts = nn.Linear(hidden_size, 1) # What this would yield is logarithm of expected value of poisson distribution

    def forward(self, input, hidden):
        input = input.view(1,1,-1)
        output, hidden = self.gru(input, hidden)
        #print(output.shape)
        output_classification = self.out_classification(output[0])
        output_counts = self.out_counts(output[0])
        
        return output_classification, output_counts, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=DEVICE)

In [424]:
def load_model(path: str):
    model = DecoderRNN(256, dataset.tensor_size)
    checkpoint = torch.load(path)
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    
    return model.to(DEVICE)

In [427]:
def train(input, counts, decoder, decoder_optimizer, criterion_class, criterion_count, dataset, acceptance_level = 0.5):
    #Input already contains start and end tokens!
    decoder_optimizer.zero_grad()
    
    target_length = input.shape[0]
    if target_length == 0:
        print("Problem!")
    #print(target_length)
    loss_class = 0
    loss_count = 0
    
    use_teacher_forcing = True if random.random() < TEACHER_FORCING_RATIO else False
    
    decoder_input = torch.cat((input[0], counts[0].view(1,-1)), dim=1) #merging first chord and number of it's ocurences
    #print(decoder_input.shape)
    decoder_hidden = decoder.initHidden()
    if use_teacher_forcing:
        #print("TEACHER FORCING")
        for i in range(1, target_length):
            decoder_output_class, decoder_output_count, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            # print(f"GIVEN INPUT: {dataset.tensor_to_token(decoder_input.detach())}")
            # print(f"DECODER OUTPUT: {dataset.tensor_to_token((decoder_output_class > acceptance_level).type(dtype=torch.float32).detach())}")
            
            loss_class += criterion_class(decoder_output_class, input[i]) # Based on i-1 chord does model predicted ith chord?
            loss_count += criterion_count(decoder_output_count, counts[i])
            
            decoder_input = torch.cat((input[i], counts[i].view(1,-1)), dim=1) # i ths chord is input to the next iteration
        loss_class.backward(retain_graph=True) #During first iteration I retain computational graph to be able to do backward once more
        loss_count.backward() #Now I cen delete the graph 
    else:
        #print("NOT TEACHER FORCING")
        for i in range(1, target_length):
            decoder_output_class, decoder_output_count, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            
            # print(f"GIVEN INPUT: {dataset.tensor_to_token(decoder_input.detach())}")
            # print(f"DECODER OUTPUT: {dataset.tensor_to_token((decoder_output_class > acceptance_level).type(dtype=torch.float32).detach())}")
            
            loss_class += criterion_class(decoder_output_class, input[i]) # Based on i-1 chord does model predicted ith chord?
            
            loss_count += criterion_count(decoder_output_count, counts[i])
            
            
            chords_predicted = (decoder_output_class > acceptance_level).type(dtype=torch.float32).detach()
            decoder_input = torch.cat((chords_predicted, counts[i]), dim=1) 
            
        loss_class.backward()
        loss_count.backward() #Now I cen delete the graph   
            #THERE also is a problem with this loss_count, since it sometimes may be very big. And network became unstable in a moment!
            #Since this is propated through entire network entire network became a trash in a moment
            
            
            #This is problably not the best idea, let it go through whole target. Not to stop too early
            #if decoder_input[0, dataset.word2num[END_TOKEN]] == 1:
            #    break
            
    #print(loss_class, loss_count)
    #loss = loss_class + loss_count #! WE CAN'T SUM UM THESE LOSSES Loss_count basically can take as big negative numbers as we wish!!!!!
    #loss_class.backward(retain_graph=True) #During first iteration I retain computational graph to be able to do backward once more
    #loss_count.backward() #Now I cen delete the graph
    if torch.isnan(loss_class):
        print("NAN!! CLASS")
    #if torch.isnan(loss_count):
    #    print("NAN!! COUNT")
    decoder_optimizer.step()

    return loss_class.item() / target_length,0#, loss_count.item() / target_length

In [413]:
def train_epoch(dataloader, decoder, decoder_optimizer, criterion_class, criterion_count,dataset):
    loop = tqdm(dataloader, leave=True)
    losses_class = []
    losses_count = []
    for batch_idx, (input, count) in enumerate(loop):
        input, count = input.to(DEVICE), count.to(DEVICE)
        #print(input.shape)
        loss_class, loss_count = train(input[0], count[0], decoder, decoder_optimizer, criterion_class, criterion_count,dataset)
        losses_class.append(loss_class)
        losses_count.append(loss_count)
        # update tqdm loop
        loop.set_postfix(loss_class=loss_class, loss_count=loss_count)
        #break
        
    #print(losses_class)
    #print(losses_count)
    return sum(losses_class) / len(losses_class), sum(losses_count)/ len(losses_count)

In [415]:
path = "../lstm_dataset_compressed_very_small.txt"

dataset = midiTextGeneratorDataset(path)
dataloader = DataLoader(dataset, batch_size=1,
                shuffle=True)

In [416]:
decoder = DecoderRNN(256, dataset.tensor_size)
decoder.to(DEVICE)

criterion_class = torch.nn.BCEWithLogitsLoss(pos_weight=torch.full((dataset.tensor_size,), POSITIVE_WEIGHT, device=DEVICE)) # This loss combines a Sigmoid layer and the BCELoss in one single class
#Which is better for numerical stability.
criterion_count = torch.nn.PoissonNLLLoss(log_input=True) # So this loss expect log(lambda) = x * b. Then it transforms it using exp
learning_rate = 0.001
optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)

In [417]:
for i in range(50):
    loss_class, loss_count =train_epoch(dataloader, decoder, optimizer, criterion_class, criterion_count,dataset)
    print(f"Epoch {i}, loss_class: {loss_class}, loss_count: {loss_count}")
    # torch.save({
    #         'epoch': i,
    #         'model_state_dict': decoder.state_dict(),
    #         'optimizer_state_dict': optimizer.state_dict(),
    #         'loss': 0,
    #         }, f"model_{i}.pt")

100%|██████████| 23/23 [00:01<00:00, 15.10it/s, loss_class=1.77, loss_count=0]


Epoch 0, loss_class: 2.3976305463973953, loss_count: 0.0


100%|██████████| 23/23 [00:01<00:00, 15.29it/s, loss_class=1.91, loss_count=0]


Epoch 1, loss_class: 2.2592174983187103, loss_count: 0.0


100%|██████████| 23/23 [00:01<00:00, 15.46it/s, loss_class=1.67, loss_count=0]


...

Epoch 8, loss_class: 1.715799369111969, loss_count: 0.0


100%|██████████| 23/23 [00:01<00:00, 14.49it/s, loss_class=1.51, loss_count=0]


Epoch 9, loss_class: 1.7119448720091064, loss_count: 0.0


 26%|██▌       | 6/23 [00:00<00:01, 11.69it/s, loss_class=2.29, loss_count=0]


KeyboardInterrupt: 

In [405]:
#-200*math.log(200)+200 # This loss behaves weirdly a little bit. Since it doesn't get to 0 when prediction is correct but rather takes negative value! However it takes minimum value for the correct solution
#Basically we are doing good. So we are minimizing this. However this doesn't have minimum equal to 0! So we can't sum up loss from classification and Poisson regression


In [418]:
#dataset = midiTextGeneratorDataset(path)

#decoder = load_model("model_8.pt")
song = "7LOSUX"
decoder_input = dataset.token_to_tensor(song).to(DEVICE)
decoder_input = torch.cat((decoder_input[0], torch.tensor([1], device = DEVICE, dtype=torch.float32).view(1,-1)), dim=1)
song += " 1"
finished = False
decoder_hidden = decoder.initHidden().to(DEVICE)
i = 0
prob = 0.5
decoder.eval()
while not finished:
    decoder_token, decoder_count, decoder_hidden = decoder(
        decoder_input, decoder_hidden)
    
    decoder_token = torch.sigmoid(decoder_token)
    #print(decoder_count)
    #print(decoder_token)
    #print(decoder_output)
    decoder_token = (decoder_token > prob).type(dtype=torch.float32)
    #print(decoder_token)
    i+=1
    #break
    token = dataset.tensor_to_token(decoder_token)

    decoder_input = decoder_token # i ths chord is input to the next iteration
        
    # if i == 1:
    #     song = token
    # else:
    song += " " + token
    
    #print(decoder_count[0])
    lambda_ = np.exp(decoder_count[0].item())
    repetitions = max([1, np.random.poisson(lambda_,1)])
    
    song += " " + str(int(repetitions))
    # if i % 10 == 0:
    #     decoder_input = dataset.token_to_tensor("").to(DEVICE)
    if i % 30 == 0:
         decoder_hidden = decoder.initHidden().to(DEVICE)
    if i == 200:
        break
    
    decoder_input = torch.cat((decoder_input, torch.tensor([repetitions], device = DEVICE, dtype=torch.float32).view(1,-1)), dim=1)
    #decoder_input = decoder_output # i ths chord is input to the next iteration
    


In [419]:
song

'7LOSUX 1 8ICO=:TU0EG 1 8ICO=:TU0EG 1 8ICO=:TU0EG 1 8ICO=:TU0EG 1 8ICO=:TU0EG 1 8ICO=:TU0EG 1 8ICO=:TU0EG 2 8ICO=:TU0EG 1 8ICO=:TU0EG 1 8ICO=:TU0EG 1 8ICO=:TU0EG 1 8ICO=:TU0EG 1 8ICO=:TU0EG 2 8ICO=:TU0EG 1 8ICO=:TU0EG 1 8ICO=:TU0EG 1 8ICO=:TU0EG 3 8ICO=:TU0EG 1 8ICO=:TU0EG 2 8ICO=:TU0EG 1 8ICO=:TU0EG 1 8ICO=:TU0EG 1 8ICO=:TU0EG 1 8ICO=:TU0EG 1 8ICO=:TU0EG 1 8ICO=:TU0EG 1 8ICO=:TU0EG 1 8ICO=:TU0EG 1 8ICO=:TU0EG 3 8ICO=:TU0EG 1 8ICO=:TU0EG 2 8ICO=:TU0EG 1 8ICO=:TU0EG 1 8ICO=:TU0EG 1 8ICO=:TU0EG 1 8ICO=:TU0EG 1 8ICO=:TU0EG 2 8ICO=:TU0EG 2 8ICO=:TU0EG 2 8ICO=:TU0EG 1 8ICO=:TU0EG 1 8ICO=:TU0EG 1 8ICO=:TU0EG 1 8ICO=:TU0EG 1 8ICO=:TU0EG 1 8ICO=:TU0EG 1 8ICO=:TU0EG 1 8ICO=:TU0EG 1 8ICO=:TU0EG 2 8ICO=:TU0EG 1 8ICO=:TU0EG 2 8ICO=:TU0EG 1 8ICO=:TU0EG 1 8ICO=:TU0EG 4 8ICO=:TU0EG 1 8ICO=:TU0EG 1 8ICO=:TU0EG 1 8ICO=:TU0EG 1 8ICO=:TU0EG 1 8ICO=:TU0EG 2 8ICO=:TU0EG 1 8ICO=:TU0EG 1 8ICO=:TU0EG 1 8ICO=:TU0EG 1 8ICO=:TU0EG 3 8ICO=:TU0EG 1 8ICO=:TU0EG 1 8ICO=:TU0EG 1 8ICO=:TU0EG 1 8ICO=:TU0EG 2 8ICO=:TU0E

In [127]:
from midiToTxt import converter, compressor

In [128]:
song = compressor.decompress(song)

In [129]:
song

c = converter.MidiTxtConverter()

track =c.string_to_multitrack(song.strip())

pypianoroll.write("lstm3.mid", track)